In [2]:
import numpy as np
import pickle
from hashlib import sha256
import threading
import sys
import socket
import time
import socketserver

In [3]:
class partial_block:
    def __init__(self, parenthash, height):
        self.parenthash = sha256(pickle.dumps(int(0))).hexdigest()
        self.txs = []
        self.height = height
    def blockhash(self):
        return sha256(pickle.dumps(self)).hexdigest()

In [4]:
class block:
    def __init__(self, nonce, partial_block):
        self.nonce = sha256(pickle.dumps(int(0))).hexdigest()
        self.partial_block = partial_block

In [5]:
class blockchain:
    def __init__(self, genesis_hash):
        self.blocks = {} # Block:Parent
        self.blockheight = {} # Blockhash: height 
        self.blocks[genesis_hash] = 0
        self.head = genesis_hash # Hash of the head block
        self.longest_chain = [genesis_hash] #List of blockhashes on the longest chain
        self.len = 1
        self.tx_included = {} #TXID: TX at the head
        self.blockdb = {} #Blockhash to block
        self.state_map = {gensis_hash:{}} #Block hash to state mapping
    def insert(self, block):
        blockhash = block.partial_block.blockhash()
        parenthash = block.partial_block.parenthash
        if parenthash not in self.blocks.keys():
            return "orphan_block"
        height = block.partial_block.height
        # Update Statemap
        temp_state = self.state_map[parenthash].copy()
        (temp_state,validity_bool) = stf(temp_state,block.partial_block.txs)
        if not validity_bool:
            return "invalid_block"
        self.state_map[blockhash] = temp_state
        #Updatestatemap complete
        self.blocks[blockhash] = parenthash
        self.blockheight[blockhash] = height
        self.blockdb[blockhash] = block
        if self.head == parenthash: # The normal scenario
            self.head = blockhash
            self.len = self.len + 1
            self.longest_chain.append(blockhash)
            #update tx_included
            for tx in block.partial_block.txs:
                self.tx_included[tx.txid] = tx
            #update tx_included complete
        elif height > self.len: #CAUTION: Do not do blockchain.insert for orphan blocks
            connected_ancestor = parenthash
            self.head = blockhash
            self.len = height
            # Now update the longest chain
            new_suffix = [parenthash,blockhash]
            while connected_ancestor not in self.longest_chain:
                connected_ancestor = self.blocks[connected_ancestor] # Look for previous generation
                new_suffix = [connected_ancestor] + new_suffix
            ca_height = self.blockheight[connected_ancestor]
            removed_suffix = self.longest_chain[(ca_height-1):]
            self.longest_chain = self.longest_chain[:(ca_height-1)]
            self.longest_chain = self.longest_chain + new_suffix
            #update tx_included
            for removed_bh in removed_suffix:
                removed_blk = self.blockdb[removed_bh]
                for tx in removed_blk.partial_block.txs:
                    del(self.tx_included[tx.txid])
            for added_bh in new_suffix:
                added_blk = self.blockdb[added_bh]
                for tx in added_blk.partial_block.txs:
                    self.tx_included[tx.txid] = tx
            #update tx_included complete
        else:
            pass
        return "block_processed"
    def head_state(self):
        return self.state_map[self.head]

In [22]:
mining_block = partial_block(sha256(pickle.dumps(int(0))).hexdigest(),2)
mining_block.parenthash = sha256(pickle.dumps(int(1))).hexdigest()
mining_block.txs.append("TX1")
print(mining_block)
print(mining_block.blockhash())
text = str(mining_block.blockhash())
print(text)
print(type(str(mining_block.parenthash)))

f8b382a39a28a91a696fc22f03c2cb7a25421035703829db619536117bbec961
f8b382a39a28a91a696fc22f03c2cb7a25421035703829db619536117bbec961
<class 'str'>


In [7]:
state = {} # Latest local state
state_map = {} 
blockdb = {} #Block hash to block    

In [19]:
# Communication handling
print(mining_block.txs)
print(mining_block.parenthash)
text_block = pickle.dumps(mining_block,0)
block_message = f"BLOCK "
end_line_encode = "\n".encode()
print(end_line_encode)

encoded_block_message = block_message.encode() + text_block
decoded_block_message = encoded_block_message.decode() # Decoded block message looks weird after line_sp[0]
print(encoded_block_message)
print(decoded_block_message)
line_sp = decoded_block_message.split(' ')
print(line_sp[0])
decoded_mining_block = pickle.loads(decoded_block_message.encode()[6:])
print(decoded_mining_block.txs)
print(decoded_mining_block.parenthash)

['TX1']
4d672a9db795f0105d536626cd190c28ed199cc8d145e2349e5457f7ff6f38be
b'\n'
b'BLOCK ccopy_reg\n_reconstructor\np0\n(c__main__\npartial_block\np1\nc__builtin__\nobject\np2\nNtp3\nRp4\n(dp5\nVparenthash\np6\nV4d672a9db795f0105d536626cd190c28ed199cc8d145e2349e5457f7ff6f38be\np7\nsVtxs\np8\n(lp9\nVTX1\np10\nasVheight\np11\nL2L\nsb.'
BLOCK ccopy_reg
_reconstructor
p0
(c__main__
partial_block
p1
c__builtin__
object
p2
Ntp3
Rp4
(dp5
Vparenthash
p6
V4d672a9db795f0105d536626cd190c28ed199cc8d145e2349e5457f7ff6f38be
p7
sVtxs
p8
(lp9
VTX1
p10
asVheight
p11
L2L
sb.
BLOCK
['TX1']
4d672a9db795f0105d536626cd190c28ed199cc8d145e2349e5457f7ff6f38be


In [25]:
#Playground
def state_change(d):
    internal = d.copy()
    internal["foo"] = "bar"
    return internal
a = {1:2,3:4}
b = state_change(a)
print(2 not in a)
b[1]=3
print(b)
a = [1,2,3]
for i in a:
    a.remove(i)
print(a)
print(a)
print(sha256(pickle.dumps(int(0))).hexdigest())

True
{1: 3, 3: 4, 'foo': 'bar'}
[2]
[2]
24c04d87a97bdcefc011d2dd182eba1beaedfe91ae22e632ffb73e5ae7086bed


In [10]:
class orphan_buffer:
    def __init__(self):
        self.blocks_to_parent = {} #Blockhash : parent
        self.blockdb = {} #Blockhash:Block
        self.dependents = {} # Parent: [list of blocks(sons)]
    def check_dependency(self, parenthash):
        if parenthash in self.dependents.keys():
            return self.dependents[parenthash]
        else:
            return []
    def insert(self, block):
        blockhash = block.partial_block.blockhash()
        parenthash = block.partial_block.parenthash
        self.blocks_to_parent[blockhash] = parenthash
        self.blockdb[blockhash] = block
        if parenthash not in self.dependents:
            self.dependents[parenthash] = [blockhash]
        else:
            self.dependents[parenthash].append(blockhash)    

In [11]:
class mempool:
    def __init__(self):
        self.transactions = [] #Need list since we need order
    def update_mempool_vanilla(self,block):
        block_txs = block.partial_block.txs
        self.transctions = [tx for tx in self.transactions if tx not in block_txs] #Remove transactions in block
    def update_mempool_state(self,state,tx_included):
        new_state = state.copy() #important to do dict copy
        self.transctions = [tx for tx in self.transactions if tx not in tx_included] #Remove transactions previously included
        for tx in self.transactions:
            (new_state, validity_bool) = stf(new_state, [tx])
            if not validity_bool:
                self.transactions.remove(tx)
    def insert(self,tx): # inserts unvalidated transactions, need to run update_mempool_state before assembling block
        self.transactions.append(tx)
    def get_tx(self,state,tx_included):
        self.update_mempool_state(state,tx_included)
        extracted_tx = self.transactions[:(min(2000,len(self.transactions)))]
        return extracted_tx
        

In [12]:
def stf(state, txs):
    temp_state = state.copy()
    valid_bool = True
    for tx in txs:
        if tx.withdraw==0:
            if tx.deposit in temp_state.keys():
                temp_state[tx.deposit] = temp_state[tx.deposit] + tx.amount
            else:
                temp_state[tx_deposit] = tx.amount
        else:    
            if temp_state[tx.withdraw] < tx.amount:
                valid_bool = False
            else:
                temp_state[tx.withdraw] = temp_state[tx.withdraw] - tx.amount
                if tx.deposit in temp_state.keys():
                    temp_state[tx.deposit] = temp_state[tx.deposit] + tx.amount
                else:
                    temp_state[tx_deposit] = tx.amount
            
    return (temp_state, valid_bool)  

In [13]:
class transaction:
    def __init__(self, txid, withdraw, deposit, amount):
        self.txid = txid
        self.withdraw = withdraw
        self.deposit = deposit
        self.amount = amount

In [14]:
# Network Worker
rcvd_bh_buffer = {}
def process_block(rcvd_blk, rcvd_bh):
    chain_msg = chain.insert(rcvd_blk)
    if chain_msg == "block_processed":
        dependencies = orphan_buffer.check_dependency(rcvd_bh)
        for orp_bh in dependencies:
            orp_blk = orphan_buffer.blockdb[orp_bh]
            process_block(orp_blk,orp_bh)
            del(orphan_buffer.blocks_to_parent[orp_bh])
            del(orphan_buffer.blockdb[orp_bh])
        del orphan_buffer.dependents[rcvd_bh]
    if chain_msg == "orphan_block":
        orphan_buffer.insert(rcvd_blk)    
    return None

def form_block():
    height = chain.len + 1
    parenthash = chain.head
    partial_blk = partial_block(parenthash,height)
    partial_blk.txs = mempool.get_tx(chain.head_state(), chain.tx_included)
    return partial_blk

rcvd_bh = rcvd_blk.partial_block.blockhash()
rcvd_bh_buffer[rcvd_bh] = None
if (rcvd_bh not in chain.blocks) and (rcvd_bh not in orphan_buffer.blocks_to_parent) and (rcvd_bh not in rcvd_bh_buffer):
    process_block(rcvd_blk, rcvd_bh)
    form_block()


NameError: name 'rcvd_blk' is not defined

In [23]:
def block_serialize(block): # Block + blockhash + nonce + parenthash + height + [txs]
    line = "BLOCK"
    line = line + " " + str(block.partial_block.blockhash()) + " " + str(block.nonce) 
    line = line + " " + str(block.partial_block.parenthash) + " " + str(block.partial_block.height)
    for tx in block.partial_block.txs:
        line = line + " " + str(tx.time) + " " + str(tx.txid) + " " + str(tx.withdraw)
        line = line + " " + str(tx.deposit) + " " + str(tx.amount)
    return line

def block_deserialize(line):
    split = line.split(" ")
    len_split = len(split)
    blockhash = split[1]
    nonce = split[2]
    parenthash = split[3]
    height = int(split[4])
    tx_list = []
    num_tx = (len_split - 5)/5
    for i in range(1,(num_tx+1)):
        tx = Transaction(split[5*i+0], split[5*i+1], int(split[5*i+2]), int(split[5*i+3]), int(split[5*i+4]))
        tx_list.append(tx)
    partial_block = partial_block(parenthash,height)
    partial_block.txs = tx_list
    block = block(nonce,partial_block)
    return block

In [28]:
def block_deserialize(line): #Pass with BLOCK
    split = line.split(" ")
    len_split = len(split)
    blockhash = split[1]
    nonce = split[2]
    parenthash = split[3]
    height = int(split[4])
    miner_ip = split[5]
    miner_port = int(split[6])
    tx_list = []
    num_tx = int(round((len_split - 7)/5))
    for i in range(1,(num_tx+1)):
        tx = Transaction(split[2+5*i+0], split[2+5*i+1], int(split[2+5*i+2]), int(split[2+5*i+3]), int(split[2+5*i+4]))
        tx_list.append(tx)
    partial_blk = partial_block(parenthash,height,miner_ip,miner_port)
    partial_blk.txs = tx_list
    blk = block(nonce,partial_blk)
    return blk

def block_serialize(block): # Block + blockhash + nonce + parenthash + height + my_ip + my_port + [txs]
    line = "BLOCK"
    line = line + " " + str(block.partial_block.blockhash()) + " " + str(block.nonce) 
    line = line + " " + str(block.partial_block.parenthash) + " " + str(block.partial_block.height)
    line = line + " " + str(block.partial_block.miner_id[0]) + " " + str(block.partial_block.miner_id[1])
    for tx in block.partial_block.txs:
        line = line + " " + str(tx.time) + " " + str(tx.txid) + " " + str(tx.withdraw)
        line = line + " " + str(tx.deposit) + " " + str(tx.amount)
    return line

print(type(time.time()))

<class 'float'>
